In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import pymongo

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw
from climatools.cliradlw.utils import mapband_new2old

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *

import IPython.display as display

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
def make_query(param=None):
    return {'param.' + name: value for name, value in param.items()}   

In [4]:
def show_html(s): display.display(display.HTML(s))
    
def show_markdown(s): display.display(display.Markdown(s))

In [5]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [6]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    dims = ['igg', 'g', 'i']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')     
    return ds['coolrg']

def pltdata_cooling_1():
    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(DS_COOL.sel(i=1)),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(DS_COOL_CRD.sum('g')),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(DS_COOL_WGT.sel(igg=10).sum('g')),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data

def nice_xlims(pltdata=None, prange=None):
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)

def plt_cool(pltdata=None):
    plin = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='linear',
                                  prange=(50, 1050))
    plin.plot_width, plin.plot_height = 400, 500
    plog = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='log',
                                  prange=(.01, 200))
    plog.plot_width, plog.plot_height = 400, 500
    everything = gridplot(plin, plog, ncols=2)
    return everything
    
def script_plt_cooling():
    pltdata = pltdata_cooling_1()
    p = plt_cool(pltdata=pltdata)
    show(p)
    show_markdown('''*FIGURE.* Cooling rate profiles.''')

In [7]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels: ds = ds.isel(pressure=ilevels)

    dims = ['i', 'igg', 'g']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df

def tbdata_flux():
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    atmpro = PARAM['atmpro']
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(DS_FLUX_CRD.sum('g'), ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10',
            'df': fmt_flux(DS_FLUX_WGT.sum('g').sel(igg=10), 
                           ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(DS_FLUX.sel(i=1), ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data

def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        show_markdown(f'''*TABLE.*  Fluxes. {d['label']}''')
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        show_markdown(
            f'''*TABLE.*  Fluxes.  ({d['label']}) - ({d0['label']})''')
        
    show_tb(benchmark)
    for d in others: show_tb(d)
    for d in others: show_tbdiff(d=d, d0=benchmark)
        
def script_tb_flux():
    data = tbdata_flux()
    tb_flux(tbdata=data)

In [8]:
def gasbands():
    bmap = mapband_new2old()
    gases = [gas for gas, _ in PARAM['molecule'].items()] 
    bands = [bmap[b] for b in PARAM['band']]
    return list(itertools.product(gases, bands))

def print_bestfit_params():
    df = pd.DataFrame()
    for gas, band in gasbands():
        param = bestfits.kdist_params(molecule=gas, band=band)
        srs = pd.Series(param)
        df[f'({gas}, {band})'] = srs
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df.sort_index())
    pd.reset_option('display.max_colwidth')
        
def print_input_param():
    df_clirad = pd.Series(PARAM, name='clirad run').to_frame()
    df_lblnew = pd.Series(PARAM_LBLNEW, name='lblnew run compared against').to_frame()
    df = pd.merge(df_clirad, df_lblnew, how='outer', 
                  left_index=True, right_index=True)
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df)
    pd.reset_option('display.max_colwidth')
        
def print_lblnew_param():
    pd.set_option('display.max_colwidth', -1)
    display.display(pd.Series(PARAM_LBLNEW).to_frame())
    pd.reset_option('display.max_colwidth')

In [9]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD.sum('g'), DS_COOL.sel(i=1)]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [10]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD.sum('g'), DS_FLUX.sel(i=1)]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [11]:
def script_fluxcoolr_excelfile():
    '''
    Save fluxes and cooling rates by layer by g-interval in excel file
    and make it available for download.
    '''
    fname = 'output_byg.xlsx'
    
    writer = pd.ExcelWriter(fname)
    
    for g in DS_FLUX_WGT.coords['g']:
        df = DS_FLUX_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'flux g={int(g)}')
        
    for g in DS_COOL_WGT.coords['g']:
        df = DS_COOL_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'cool g={int(g)}')    

In [12]:


    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()

In [13]:
%run param.py

client = pymongo.MongoClient('localhost', 27017)

doc = client.lblnew.bestfit_lw.find_one(make_query(param=PARAM_LBLNEW))
DS_FLUX_CRD = load_output_file(io.StringIO(doc['output_fluxg']))
DS_COOL_CRD = load_output_file(io.StringIO(doc['output_coolrg']))
DS_FLUX_WGT = load_output_file(io.StringIO(doc['output_wfluxg']))
DS_COOL_WGT = load_output_file(io.StringIO(doc['output_wcoolrg']))

doc = client.cliradnew.lw.find_one(make_query(param=PARAM))
DS_FLUX = load_output_file(io.StringIO(doc['output_flux']))
DS_COOL = load_output_file(io.StringIO(doc['output_coolr']))

In [14]:
script()
if 'ng_refs' in PARAM_LBLNEW:
    script_fluxcoolr_excelfile()

### CLIRAD-LW

### Table of Contents

# Input Parameters

,clirad run,lblnew run compared against
atmpro,trp,trp
band,[1],1
commitnumber,1013f91,5014a19
conc,-,-
dv,-,0.001
klin,-,0
molecule,{'h2o': 'atmpro'},h2o
ng_adju,-,"[0, 0, 0]"
ng_refs,-,"[3, 3, 6]"
nv,-,1000


# Best-fit Parameters

...

*TABLE.*  Fluxes.  (CLIRAD) - (CRD)

# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312        1  5.484453                    1  3.202201
0.000750        2  4.371989                    2  3.028921
0.001052        3  3.964642                    3  2.948787
0.001476        4  3.533565                    4  2.827812
0.002070        5  3.096668                    5  2.656999
0.002904        6  2.673518                    6  2.430584
0.004074        7  2.278111                    7  2.149955
0.005714        8  1.917874                    8  1.828019
0.008015        9  1.594254                    9  1.492561
0.011243       10  1.306455                   10  1.183929
0.015771       11  1.053567                   11  0.938181
0.022122       12  0.833762                   12  0.762538
0.031031       13  0.641448                   13  0.630862
0.043528       14  0.465204                   14  0.500740
0.061057       15  0.373366                   15  0.415467
0.085645       16  0.371362                   16  0.387412
0.120136       17  0.376996                   17  0.361741
0.168516       18  0.375093                   18  0.342087
0.236378       19  0.379240                   19  0.348322
0.331549       20  0.395566                   20  0.377399
0.465100       21  0.433125                   21  0.420723
0.652400       22  0.498422                   22  0.480736
0.915100       23  0.516842                   23  0.507476
1.283650       24  0.496543                   24  0.515046
1.800600       25  0.479665                   25  0.524832
2.525700       26  0.452415                   26  0.494559
3.542800       27  0.427615                   27  0.458426
4.969550       28  0.405631                   28  0.413945
6.970850       29  0.383934                   29  0.370653
9.778100       30  0.361998                   30  0.348334
13.715850      31  0.339606                   31  0.334558
19.239350      32  0.316376                   32  0.308482
26.987250      33  0.292421                   33  0.288024
37.855300      34  0.270062                   34  0.269246
53.100050      35  0.228822                   35  0.222799
73.887500      36  0.160603                   36  0.149784
97.662500      37  0.101895                   37  0.085427
121.437500     38  0.111572                   38  0.091352
145.212500     39  0.164947                   39  0.144183
168.987500     40  0.218429                   40  0.198168
192.762500     41  0.346566                   41  0.323969
216.537500     42  0.702389                   42  0.677461
240.312500     43  1.040207                   43  1.013820
264.087500     44  1.251138                   44  1.282455
287.862500     45  1.300838                   45  1.346437
311.637500     46  1.226337                   46  1.222849
335.412500     47  1.093298                   47  1.145683
359.187500     48  0.932852                   48  0.981984
382.962500     49  0.738241                   49  0.736520
406.737500     50  0.577737                   50  0.580315
430.512500     51  0.434766                   51  0.457623
454.287500     52  0.309290                   52  0.321208
478.062500     53  0.205649                   53  0.190346
501.837500     54  0.127220                   54  0.095649
525.612500     55  0.074217                   55  0.044915
549.387500     56  0.042028                   56  0.022834
573.162500     57  0.024446                   57  0.014539
596.937500     58  0.015662                   58  0.011013
620.712500     59  0.012004                   59  0.009708
644.487500     60  0.012191                   60  0.010860
668.262500     61  0.016119                   61  0.015286
692.037500     62  0.021770                   62  0.021241
715.812500     63  0.022372                   63  0.021969
739.587500     64  0.014486                   64  0.014314
763.362500     65  0.003251                   65  0.003182
78

# Fluxes by Layer

CRD                           CLIRAD                \
                        flug       fldg      fnetg       flug          fldg   
pressure    level                                                             
0.000000    1     -34.370336   0.000000 -34.370336 -34.159526 -6.985908e-07   
0.000624    2     -34.370353   0.000423 -34.369930 -34.159542  2.523225e-04   
0.000876    3     -34.370360   0.000560 -34.369800 -34.159548  3.489977e-04   
0.001229    4     -34.370369   0.000735 -34.369634 -34.159557  4.806170e-04   
0.001723    5     -34.370381   0.000954 -34.369428 -34.159568  6.573512e-04   
0.002417    6     -34.370396   0.001223 -34.369173 -34.159582  8.901111e-04   
0.003391    7     -34.370415   0.001551 -34.368865 -34.159600  1.189109e-03   
0.004757    8     -34.370440   0.001944 -34.368496 -34.159625  1.561374e-03   
0.006672    9     -34.370473   0.002412 -34.368061 -34.159657  2.008325e-03   
0.009359    10    -34.370516   0.002963 -34.367553 -34.159700  2.525965e-03   
0.013128    11    -34.370574   0.003603 -34.366970 -34.159757  3.111691e-03   
0.018415    12    -34.370653   0.004342 -34.366310 -34.159835  3.777831e-03   
0.025830    13    -34.370764   0.005186 -34.365579 -34.159947  4.559116e-03   
0.036232    14    -34.370932   0.006145 -34.364787 -34.160114  5.503779e-03   
0.050823    15    -34.371226   0.007242 -34.363984 -34.160399  6.654217e-03   
0.071291    16    -34.371692   0.008614 -34.363078 -34.160844  8.107246e-03   
0.100000    17    -34.372302   0.010486 -34.361816 -34.161417  9.997736e-03   
0.140271    18    -34.373033   0.013016 -34.360017 -34.162094  1.239999e-02   
0.196760    19    -34.373896   0.016389 -34.357507 -34.162874  1.547016e-02   
0.275997    20    -34.374880   0.020932 -34.353948 -34.163739  1.960401e-02   
0.387100    21    -34.375919   0.027178 -34.348741 -34.164601  2.543415e-02   
0.543100    22    -34.376727   0.035989 -34.340738 -34.165151  3.375926e-02   
0.761700    23    -34.376334   0.048503 -34.327831 -34.164434  4.549193e-02   
1.068500    24    -34.374251   0.065202 -34.309047 -34.161956  6.145932e-02   
1.498800    25    -34.370666   0.086928 -34.283738 -34.157872  8.363050e-02   
2.102400    26    -34.365195   0.115754 -34.249441 -34.151767  1.150556e-01   
2.949000    27    -34.357799   0.153728 -34.204070 -34.143628  1.565189e-01   
4.136600    28    -34.348117   0.204203 -34.143914 -34.133052  2.104411e-01   
5.802500    29    -34.335450   0.271582 -34.063867 -34.119448  2.785324e-01   
8.139200    30    -34.318946   0.361351 -33.957596 -34.101870  3.635606e-01   
11.417000   31    -34.297622   0.480583 -33.817039 -34.079424  4.763802e-01   
16.014700   32    -34.270434   0.638354 -33.632079 -34.051051  6.302359e-01   
22.464000   33    -34.236403   0.846024 -33.390379 -34.015881  8.307594e-01   
31.510500   34    -34.194318   1.117305 -33.077014 -33.972692  1.096256e+00   
44.200100   35    -34.138628   1.467563 -32.671064 -33.916547  1.444876e+00   
62.000000   36    -34.085660   1.897073 -32.188586 -33.865221  1.863374e+00   
85.775000   37    -34.116820   2.380544 -31.736276 -33.904031  2.324067e+00   
109.550000  38    -34.281788   2.832480 -31.449307 -34.080842  2.741493e+00   
133.325000  39    -34.511724   3.376639 -31.135084 -34.321556  3.239511e+00   
157.100000  40    -34.726650   4.056110 -30.670541 -34.544215  3.868278e+00   
180.875000  41    -34.930961   4.875586 -30.055375 -34.754083  4.636307e+00   
204.650000  42    -35.155978   6.076643 -29.079335 -34.985067  5.779786e+00   
228.425000  43    -35.479706   8.378523 -27.101184 -35.317464  8.020326e+00   
252.200000  44    -35.902802  11.731171 -24.171632 -35.754104  1.131250e+01   
275.975000  45    -36.416625  15.768597 -20.648028 -36.284050  1.545462e+01   
299.750000  46    -37.006561  20.022106 -16.984455 -36.891774  1.985473e+01   
323.525000  47    -37.655800  24.125101 -13.530699 -37.559668  2.396692e+01   
347.300000  48    -38.349100  27.897475 -10.451625 -38.271114  2.790530e+01

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')